In [1]:
import sys
sys.path.insert(1, r"Q:\IlMIoDrive\magistrale\1anno\terzo_periodo\MAS\analysis")
import json as js
import bz2
import copy
import os
from threading import Thread
import pickle
#import traceback 
#import sys 

Insert path to year folder (*bz_files_path*) (containing all months folders) and saving folder (*save_path*) here. The latter must contain a subfolder for every month , called $01$, $02$ etc...

In [2]:
bz_files_path = r"D:\TwitterUsers\archiveTeam\months\2012\\" 
save_path = r"D:\TwitterUsers\archiveTeam\saved_objs_with_keywords\\"

bz2 archiveTeam parsing:

In [3]:
fldr = r"06"
listOfFiles = list()
for (dirpath, dirnames, filenames) in os.walk(bz_files_path+fldr):
    listOfFiles += [os.path.join(dirpath, file) for file in filenames]

In [4]:
len(listOfFiles)

41171

In [5]:
#listOfFiles.index('M:\\TwitterUsers\\archiveTeam\\months\\2012\\\\06\\24\\15\\25.json.bz2')

In [6]:
#listOfFiles[:10]

In [7]:
class bz2Thread(Thread):
    
    def __init__(self,name, paths, save_path, save_every = 1440):
        Thread.__init__(self)
        self.name  = name
        self.paths = paths
        self.save_path = save_path
        self.save_every = save_every
        
        
    def parse_tweet(self,tweet,path):
        result  = copy.deepcopy(tweet)
    
        try:
            for field in path:
                result = result[field]
            return result
        except KeyError:
            return []
    
    
    def run(self):
        i = 0
        j = 0
        total = len(self.paths)
        objs = []
        for path in self.paths:
            bz_file = bz2.BZ2File(path)
            line_list = bz_file.readlines()
            tweets = [js.loads(statusObj.decode('utf8').replace("\r\n","")) for statusObj in line_list]
            objs.extend([{"tweet_id":self.parse_tweet(tweet, ["id"]),"retweeted_tweet_id":self.parse_tweet(tweet,["retweeted_status","id"]),"retweeted_user_id":self.parse_tweet(tweet,["retweeted_status","user","id"]),"hashtags":[dct["text"] for dct in self.parse_tweet(tweet,["entities","hashtags"])], "tweet_created_at":self.parse_tweet(tweet,["created_at"]),"text":self.parse_tweet(tweet,["text"]),"user_mentions": [mention["id"] for mention in self.parse_tweet(tweet,["entities","user_mentions"])],"user_id":self.parse_tweet(tweet,["user","id"]),"profile_created_at":self.parse_tweet(tweet,["user","created_at"]),'followers_count': self.parse_tweet(tweet,["user","followers_count"]),'friends_count': self.parse_tweet(tweet,["user","friends_count"]), "statuses_count":self.parse_tweet(tweet, ["user","statuses_count"]), "favourites_count":self.parse_tweet(tweet, ["user","favourites_count"]) } for tweet in tweets if self.parse_tweet(tweet, ["user","lang"])=="en" ])
            i += 1
            if i%self.save_every == 0:
                with open(self.save_path+str(j)+".txt", "wb") as f:
                    pickle.dump(objs, f)
                objs = []
                j += 1
            
            if self.name == "thread0":
                print("done",i,"out of", total,end = "\r")
        
        if len(objs) > 0:
            with open(self.save_path+str(j)+".txt", "ab") as f:
                pickle.dump(objs, f)
                objs = []

In [8]:
bz2Thread1 = bz2Thread(name = "thread0",paths = listOfFiles ,save_path = save_path + fldr +"\objs_")

In [9]:
%%time
bz2Thread1.start()
bz2Thread1.join()

Wall time: 1d 40min 22s1469 out of 41171 5759 out of 41171 8654 out of 41171 10050 out of 41171 11468 out of 41171 11477 out of 41171 18456 out of 41171 22709 out of 41171 24112 out of 41171 28952 out of 41171 28966 out of 41171 28979 out of 41171 28982 out of 41171 28985 out of 41171 28998 out of 41171 29000 out of 41171 29004 out of 41171 29009 out of 41171 29049 out of 41171 29057 out of 41171 29058 out of 41171 29104 out of 41171 41171 31240 out of 41171 35462 out of 41171 35472 out of 41171


In [10]:
#'M:\\TwitterUsers\\archiveTeam\\months\\2012\\\\06\\02\\04\\16.json.bz2'

In [11]:
#'M:\\TwitterUsers\\archiveTeam\\months\\2012\\\\06\\24\\15\\25.json.bz2'

In [12]:
#bz_file =bz2.BZ2File('M:\\TwitterUsers\\archiveTeam\\months\\2012\\\\06\\24\\15\\25.json.bz2')